In [1]:
import numpy as np
from datetime import datetime

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

In [2]:
class LogicGate:
    
    def __init__(self,gateName,xdata,tdata,i_node,h1_node,h2_node,o_node,learning_rate,iteration_count):
        self.gateName = gateName
        if xdata.ndim == 1:
            self.xdata = xdata.reshape(-1,1)
            self.tdata = tdata.reshape(-1,1)
        elif xdata.ndim == 2:
            self.xdata = xdata.reshape(-1,2)
            self.tdata = tdata.reshape(-1,1)
        
        self.i_node = i_node
        self.h1_node = h1_node
        self.h2_node = h2_node
        self.o_node = o_node
        self.learning_rate = learning_rate
        self.iteration_count = iteration_count
        
        self.W2 = np.random.rand(self.i_node,self.h1_node)
        self.b2 = np.random.rand(self.h1_node)
        
        self.W3 = np.random.rand(self.h1_node,self.h2_node)
        self.b3 = np.random.rand(self.h2_node)
        
        self.W4 = np.random.rand(self.h2_node,self.o_node)
        self.b4 = np.random.rand(self.o_node)
    
    def loss_func(self):
        delta = 1e-7
        
        z2 = np.dot(self.xdata,self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2,self.W3) + self.b3
        a3 = sigmoid(z3)
        
        z4 = np.dot(a3,self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        return -np.sum(self.tdata*np.log(y+delta) + (1-self.tdata)*np.log(1-y+delta))
    
    def feed_forward(self):
        delta = 1e-7
        
        z2 = np.dot(self.xdata,self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2 , self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        z4 = np.dot(a3,self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        return -np.sum(self.tdata*np.log(y+delta) + (1-self.tdata)*np.log(1-y+delta))
    
    def predict(self,test_data):
        z2 = np.dot(test_data, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
        
        z4 = np.dot(a3,self.W4) + self.b4
        y = a4 = sigmoid(z4)
        
        if y >= 0.5:
            result = 1
        else :
            result = 0
        return y,result
    
    def accuracy1(self,test_xdata,test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val , logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        return matched_list, not_matched_list
    
    def accuracy2(self,test_data):
        self.test_xdata = test_data[:, :-1]
        self.test_tdata = test_data[:, -1]
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(self.test_xdata)):
            
            (real_val , logical_val) = self.predict(self.test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        return matched_list, not_matched_list
    
    def train(self):
        f = lambda x : self.loss_func()
        start_time = datetime.now()
        for step in range(self.iteration_count):
            self.W2 -= self.learning_rate * numerical_derivative(f,self.W2)
            self.b2 -= self.learning_rate * numerical_derivative(f,self.b2)
            
            self.W3 -= self.learning_rate * numerical_derivative(f,self.W3)
            self.b3 -= self.learning_rate * numerical_derivative(f,self.b3)
            
            self.W4 -= self.learning_rate * numerical_derivative(f,self.W4)
            self.b4 -= self.learning_rate * numerical_derivative(f,self.b4)
            
            if (step % 400) == 0 :
                print("step = ", step, "error_val = ", self.loss_func())
        end_time = datetime.now()
        print("")
        print("elapsed_time : ", end_time - start_time)

In [3]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
AND_tdata = np.array([0,0,0,1])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_1_nodes = 2
hidden_2_nodes = 2
output_nodes = 1

lr = 1e-1
count = 10001
And_obj = LogicGate("AND_Gate",xdata,AND_tdata,input_nodes,hidden_1_nodes,hidden_2_nodes,output_nodes,lr,count)
And_obj.train()

step =  0 error_val =  3.747618921057667
step =  400 error_val =  2.246492969065593
step =  800 error_val =  2.2201964590850505
step =  1200 error_val =  1.3541582159958705
step =  1600 error_val =  0.10655025239571209
step =  2000 error_val =  0.04295857311905062
step =  2400 error_val =  0.025883719137184343
step =  2800 error_val =  0.018262147513085338
step =  3200 error_val =  0.014010031742913657
step =  3600 error_val =  0.01131843222572657
step =  4000 error_val =  0.009470078661049208
step =  4400 error_val =  0.008126455153790593
step =  4800 error_val =  0.007107816589818909
step =  5200 error_val =  0.00631021044369882
step =  5600 error_val =  0.005669485621061614
step =  6000 error_val =  0.005143979308606357
step =  6400 error_val =  0.004705494275471922
step =  6800 error_val =  0.004334286140348068
step =  7200 error_val =  0.00401612880633429
step =  7600 error_val =  0.0037405198214013834
step =  8000 error_val =  0.0034995414825959364
step =  8400 error_val =  0.003

In [4]:
print(And_obj.gateName, "\n")
test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = And_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = And_obj.accuracy2(test_data)

print(true_list)
print(false_list)

AND_Gate 

Accuracy =>  1.0
Accuracy =>  1.0
[0, 1, 2, 3]
[]


In [6]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
XOR_tdata = np.array([0,1,1,0])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_1_nodes = 4
hidden_2_nodes = 4
output_nodes = 1

lr = 1e-1
count = 10001
XOR_obj = LogicGate("XOR_Gate",xdata,XOR_tdata,input_nodes,hidden_1_nodes,hidden_2_nodes,output_nodes,lr,count)
XOR_obj.train()

step =  0 error_val =  3.744912248352434
step =  400 error_val =  2.7696966108301595
step =  800 error_val =  2.7671268275228416
step =  1200 error_val =  2.762384030537747
step =  1600 error_val =  2.750606254369114
step =  2000 error_val =  2.7028166146509056
step =  2400 error_val =  2.364982936801042
step =  2800 error_val =  2.013914000704299
step =  3200 error_val =  1.9222873538399
step =  3600 error_val =  1.4995342010685626
step =  4000 error_val =  0.16219581872251387
step =  4400 error_val =  0.07542813421121904
step =  4800 error_val =  0.047549246483705204
step =  5200 error_val =  0.03377173224236853
step =  5600 error_val =  0.025676632728398867
step =  6000 error_val =  0.020435481392148958
step =  6400 error_val =  0.016815272762035788
step =  6800 error_val =  0.014192727259909534
step =  7200 error_val =  0.012221163001866585
step =  7600 error_val =  0.010694112236948446
step =  8000 error_val =  0.00948197306289448
step =  8400 error_val =  0.008499861144970687
ste

In [21]:
print(XOR_obj.gateName, "\n")
test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = XOR_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = XOR_obj.accuracy2(test_data)

print(true_list)
print(false_list)

XOR_Gate 

Accuracy =>  1.0
Accuracy =>  1.0
[0, 1, 2, 3]
[]


In [23]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
OR_tdata = np.array([0,1,1,1])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_1_nodes = 2
hidden_2_nodes = 1
output_nodes = 1

lr = 1e-1
count = 10001
OR_obj = LogicGate("OR_Gate",xdata,OR_tdata,input_nodes,hidden_1_nodes,hidden_2_nodes,output_nodes,lr,count)
OR_obj.train()

step =  0 error_val =  2.322393485152633
step =  400 error_val =  2.1355724743204045
step =  800 error_val =  0.28404117882621904
step =  1200 error_val =  0.09099127333265555
step =  1600 error_val =  0.05300457375186958
step =  2000 error_val =  0.03726051344164258
step =  2400 error_val =  0.02869543094140141
step =  2800 error_val =  0.023321287077913543
step =  3200 error_val =  0.01963817670982798
step =  3600 error_val =  0.01695770216967096
step =  4000 error_val =  0.014920050476479055
step =  4400 error_val =  0.013319009078203234
step =  4800 error_val =  0.012027968755168133
step =  5200 error_val =  0.010964919486104162
step =  5600 error_val =  0.010074414049594414
step =  6000 error_val =  0.009317625205623492
step =  6400 error_val =  0.008666555843755192
step =  6800 error_val =  0.008100510842200053
step =  7200 error_val =  0.007603863257622413
step =  7600 error_val =  0.007164592972168102
step =  8000 error_val =  0.00677330323844979
step =  8400 error_val =  0.006

In [24]:
print(OR_obj.gateName, "\n")
test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = OR_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = OR_obj.accuracy2(test_data)

print(true_list)
print(false_list)

OR_Gate 

Accuracy =>  1.0
Accuracy =>  1.0
[0, 1, 2, 3]
[]


In [25]:
xdata = np.array([[0,0],[0,1],[1,0],[1,1]])
NAND_tdata = np.array([1,1,1,0])
test_data = np.array([[0,0],[0,1],[1,0],[1,1]])

input_nodes = 2
hidden_1_nodes = 2
hidden_2_nodes = 1
output_nodes = 1

lr = 1e-1
count = 10001
NAND_obj = LogicGate("NAND_Gate",xdata,NAND_tdata,input_nodes,hidden_1_nodes,hidden_2_nodes,output_nodes,lr,count)
NAND_obj.train()

step =  0 error_val =  2.449563596902876
step =  400 error_val =  2.247215209511014
step =  800 error_val =  2.2184689367340376
step =  1200 error_val =  0.7126745141374509
step =  1600 error_val =  0.1184683550696255
step =  2000 error_val =  0.06079897015411704
step =  2400 error_val =  0.04073339292035784
step =  2800 error_val =  0.030604292814092538
step =  3200 error_val =  0.02450578905453312
step =  3600 error_val =  0.020433605520994796
step =  4000 error_val =  0.017522345151031696
step =  4400 error_val =  0.015337693344199203
step =  4800 error_val =  0.013637859746723569
step =  5200 error_val =  0.012277586350658926
step =  5600 error_val =  0.011164360779717117
step =  6000 error_val =  0.010236473949876881
step =  6400 error_val =  0.009451187366088954
step =  6800 error_val =  0.008777964611757963
step =  7200 error_val =  0.0081944072561884
step =  7600 error_val =  0.007683713427122136
step =  8000 error_val =  0.007233032691413097
step =  8400 error_val =  0.0068323

In [26]:
print(NAND_obj.gateName, "\n")
test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = NAND_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = NAND_obj.accuracy2(test_data)

print(true_list)
print(false_list)

NAND_Gate 

Accuracy =>  1.0
Accuracy =>  1.0
[0, 1, 2, 3]
[]
